In [188]:
%matplotlib 
#%matplotlib inline
#%matplotlib qt
import os
import csv
import numpy as np
import fnmatch
from pathlib import Path
import pandas as pd
import xlrd
import matplotlib.pyplot as plt
from scipy import stats 
import statsmodels
from statsmodels.graphics.gofplots import qqplot
import pickle
from itertools import *
from operator import *

import itertools
from scipy import signal

Using matplotlib backend: Qt5Agg


In [159]:
# first find the number of rows in the existing file

#subjectName = 'Per'
subjectName = 'Torben'
#subjectName = 'John'


if subjectName == 'Torben':
    # 0 indicates the trial not to be considered
    hot_indices = ['1', '0', '2', '1', '2']
else:
    hot_indices = ['1', '2', '1', '2']

filePath_PythonData = r"C:\DTU\Data\PainExperiments\201907_PainTrial2\ExportedData\raw" + "\\" + subjectName + ".xlsx"


df_data = pd.read_excel(filePath_PythonData)

In [161]:
# extract pupil data

pupilRawLeft = df_data['Pupil diameter left [mm]'].tolist()
pupilRawRight = df_data['Pupil diameter right [mm]'].tolist()



pupilRawLeft_woNan = [pupilSize for ind, pupilSize in enumerate(pupilRawLeft) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight[ind])]
pupilRawRight_woNan = [pupilSize for ind, pupilSize in enumerate(pupilRawRight) if not np.isnan(pupilSize) and not np.isnan(pupilRawLeft[ind])]

# extract time 
timeMicroS = df_data['Recording timestamp [μs]'].tolist()
timeMilliS = [time/1000 for time in timeMicroS]
timeS = [time/1000 for time in timeMilliS]

timeMilliS_woNan = [timeMilliS[ind] for ind, pupilSize in enumerate(pupilRawLeft) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight[ind])]


# extract event information
events = df_data['Event'].tolist()

# correct the event information based on the nan values removed

events_wInd = [(ind, event) for ind, event in enumerate(events) if not pd.isnull(event)]

#events_wInd = list()
#for ind, event in events_wInd0:
#    timeEvent, indexEvent = nearestTimePoint(timeMilliS_woNan, timeMilliS[ind])
#    events_wInd.append(tuple((indexEvent, event)))
    

# extract the logged events
indices_loggedEvents_new = [i for i, x in events_wInd if "Logged live Event" in x]



In [162]:
def filterBlinks_pain(pupilDataL, pupilDataR, timeInDatetime):
    # filter any blinks and nan values lasting around 250ms (on average)
    # first the single nan occurances are replaced with mean of the values on either sides, 
    # as they are assumed to be from hardware problems
    # for the rest of the blinks, 250ms before and after the nan values are interpolated with a linear function
    # returns a dataframe with pupil size, and timestamp
    # http://faculty.washington.edu/chudler/facts.html
   
    # create a dataframe from the pupilsize and time
    pupilData_df = pd.DataFrame(list(zip(timeInDatetime, pupilDataL, pupilDataR)), columns=['timeStamp', 'pupilLeft', 'pupilRight'])
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (22 samples) before and after the blink will also be removed
    extraBlinkSamples = 22   
    #extraBlinkSamples = 0
    
    # in case of single missing data, that are due to hardware error, replace with the mean of the pupil size before and
    # after the nan value
    # missing values will be the same for left and right pupil
    missingVal_Single_left = np.argwhere(np.isnan(pupilDataL)).tolist()
    missingVal_Single_right = np.argwhere(np.isnan(pupilDataR)).tolist()
    
    missingVal_Single = np.unique(np.array(sorted([index for list_ in [missingVal_Single_left, missingVal_Single_right] for index in list_])))
    
    #missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list 
    
    missingVal_Single_remove = [val for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataL)-1)) if (not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])) or ((not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])))]
    
    
    # if no blinks present, return the data
    if len(missingVal_Single) == 0:
        print('no nan values present')
        interpolatedNan = np.array([False]*len(pupilData_df['pupilLeft']))
        return pupilData_df, interpolatedNan
    
    # find the index and values to replace for single nan values
    #pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single_remove) if (val != 0 and val != (len(pupilDataL)-1)) if not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])]
    #pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single_remove) if (val != 0 and val != (len(pupilDataR)-1)) if not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])]
    
    pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single_remove)]
    pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single_remove)]
    
    
    interpolatedNan = np.array([True if ind in dict(pupilData_tuples_replaceSingleNan_left) else False for ind, val in enumerate(pupilDataL)])
    
    
    # replace the single nan values with the mean of the pupil size on either sides
    indList = -1
    for ind, val in pupilData_tuples_replaceSingleNan_left:
        #print(ind, len(pupilData_df['pupilRight']))
        indList = indList + 1
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilLeft')] = val
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilRight')] = pupilData_tuples_replaceSingleNan_right[indList][1]
        
    
    # again, find the nan values in the pupil size
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index
    # to the next nan value
    
    
    
    # find the nan values again from pupilData['pupilLeft']
    missingVal_Rest_trueFalse_left = pupilData_df['pupilLeft'].isnull()
    missingVal_Rest_trueFalse_right = pupilData_df['pupilRight'].isnull()
    missingVal_Rest_left = [i for i, x in enumerate(missingVal_Rest_trueFalse_left) if x]
    missingVal_Rest_right = [i for i, x in enumerate(missingVal_Rest_trueFalse_right) if x]
    
    
    missingVal_Rest = np.unique(np.array(sorted([index for list_ in [missingVal_Rest_left, missingVal_Rest_right] for index in list_])))
 
    #missingVal_Rest_trueFalse = sorted(missingVal_Rest_trueFalse_left + list(set(missingVal_Rest_trueFalse_right)) - set(missingVal_Rest_trueFalse_left))
    
    
    # if no blinks left, return the current pupilData
    if len(missingVal_Rest) == 0:
        print('no blinks after single nan values removed')
        return pupilData_df, interpolatedNan
    
        
    
    
    
    # in the blinks left, find when the blinks start by finding a difference in the consecutive values of the indices
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    blinkStart_tupleList = [(ind, sum(missingVal_RestDifference[0:ind+1])) for ind, val in enumerate(missingVal_RestDifference) if val != 1]
    
    blinkStart_tupleList_wLength = list()
    
    # create a list of tuples of blink start index and the length of the blink
    ind = -1
    blinkLengthSum = 0
    for blink_ind, blinkStartInd in blinkStart_tupleList:
        ind = ind + 1
        if ind != len(blinkStart_tupleList) - 1:
            
            blinkLength = blinkStart_tupleList[ind+1][0]-blink_ind
            blinkLengthSum = blinkLengthSum + blinkLength
            
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
        else:
            # for the last blink -- all blink lengths summed and subtracted from the length of the list
            # missingVal_RestDifference 
            blinkLength = len(missingVal_RestDifference)-blinkLengthSum
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
     
    # add to vector with start and end of tuple
    #beforeAfterNan = [False]*len(pupilData_df['pupilLeft'])
    #for blinkStart, blinkLength in blinkStart_tupleList_wLength:
    #    beforeAfterNan[blinkStart] = True
    #    beforeAfterNan[blinkStart+blinkLength] = True
    #    #print('start and end points: ', pupilData_df['timeStamp'][blinkStart], pupilData_df['timeStamp'][blinkStart + blinkLength])
    
    
    # create lists with start and end values for the blinks, based on blinkStart_tupleList_wLength, regardless of the blink length
    blink_missingData_startList = [blinkStartInd - extraBlinkSamples if (blinkStartInd - extraBlinkSamples) > 0 else 0 for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    blink_missingData_endList = [blinkStartInd + blinkLength + extraBlinkSamples if (blinkStartInd + blinkLength + extraBlinkSamples) < (len(pupilData_df['pupilLeft'])-1) else (len(pupilData_df['pupilLeft'])-1) for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    # create a list of tuples from the start and end points of the blink
    blink_missingData_startEndTuple = [(blinkStart, blink_missingData_endList[ind]) for ind, blinkStart in enumerate(blink_missingData_startList)] 
    
    
    # check if blinks need to be combined - blinksCombine is a list of list of 2 elements, the index of the blinks that should be combined
    blinksCombine = [[ind, ind+1] for ind, blink in enumerate(blink_missingData_startEndTuple[0:-1]) if blink[1] > blink_missingData_startEndTuple[ind+1][0]]
        
    if blinksCombine:
        print('blinks being combined')
        # combine blinks that need to be combined - if multiple consecutive blinks need to be removed: eg - [1, 2], [2, 3] 
        # are included in the blinksCombine, the combined version should be [1, 3] 
        blinksCombineFinal = list()
        ind = -1
        while ind < len(blinksCombine)-2:
            
            ind = ind + 1
            blinkCombining = blinksCombine[ind]
            blinksCombineFinal.append(blinkCombining)
            while ind < len(blinksCombine)-2 and blinkCombining[1] == blinksCombine[ind+1][0]:
                # change the ending of the last added blink of blinksCombineFinal
                blinksCombineFinal[-1][1] = blinksCombine[ind+1][1]
                ind = ind + 1
            
            
        if len(blinksCombine) == 1:
            blinksCombineFinal = blinksCombine.copy()
            
        
        if blinksCombine[-1][1] != blinksCombineFinal[-1][1]:
            if blinksCombine[-1][0] == blinksCombineFinal[-1][1]:
                blinksCombineFinal[-1][1] = blinksCombine[-1][1]
            else:
                blinksCombineFinal.append(blinksCombine[-1])
        
        
        
        
    #    for w, z in groupby(sorted(list(blinksCombine)), lambda x, y=itertools.count(): next(y)-x):
    #        group = list(z)
    #        blinksCombineFinal.append(tuple((group[0], group[-1])))
        
        for x in sorted(blinksCombineFinal, reverse=True):
            new_start = blink_missingData_startEndTuple[x[0]][0] 
            new_end = blink_missingData_startEndTuple[x[1]][1] 
            
            x_start = x[0]
            x_end = x[1]
            
            # delete also the blinkStart_tupleList_wLength, since it is going to be used to compute other metrics
            for blinkRemove in range(x[1], x[0]-1, -1):
                del blink_missingData_startEndTuple[blinkRemove]
            
            blink_missingData_startEndTuple.insert(x[0], tuple((new_start, new_end)))
    
    
    blinkAndNonBlinkDurationList = [length/90 for start, length in blinkStart_tupleList_wLength]
    
    
    # remove blinks from data
    for blinkStart, blinkEnd in blink_missingData_startEndTuple:
        pupilData_df.loc[blinkStart:blinkEnd,'pupilLeft'] = np.nan
        pupilData_df.loc[blinkStart:blinkEnd,'pupilRight'] = np.nan
        replaceTrueList = range(blinkStart, blinkEnd+1, 1)
        interpolatedNan[replaceTrueList] = True
         
    pupilData_df['pupilLeft'] = pupilData_df['pupilLeft'].astype(float).interpolate('linear', limit_direction = 'both')
    pupilData_df['pupilRight'] = pupilData_df['pupilRight'].astype(float).interpolate('linear', limit_direction = 'both')
    
    if pupilData_df.isnull().any().any():
        print('nan values in filtered data')
        #for i,val in enumerate(pupilData_filter[0:5000]):
        #    print(i, val, pupilData_woSingleMissingData[i])
        
    
    return pupilData_df, interpolatedNan

In [163]:
def hampel(dvec, radius=5, nsigma=3, rem_nomed=False):

    # replace outliers with median values (hampel filter)
    mvec = pd.Series(dvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    svec = 1.4862 * np.abs(dvec-mvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    plonk = np.abs(dvec-mvec) > nsigma*svec
    dvec = np.array(dvec)
    dvec[plonk.tolist()] = mvec[plonk.tolist()]

    # remove "bad data" where we cannot calculate a median value due to already missing values
    if (rem_nomed):
        dvec[np.isnan(mvec)] = np.nan
    return dvec


## Check filtering of pupil size using hampel filter

In [164]:
# remove nan values that occur in the other eye
#pupilLeft_equal = [val for ind, val in enumerate(pupilRawLeft_woNanPairs) if ]

# filter the pupil size for pins in the dark using hampel filter 



pupilData_woBlinks, interpolatedValues = filterBlinks_pain(pupilRawLeft, pupilRawRight, timeMilliS)

#pupilData_df_hampel = dict()
    
pupilData_filtered = pupilData_woBlinks.copy()
pupilData_filtered['pupilLeft_hampel'] = hampel(pupilData_woBlinks['pupilLeft'], 25, 3, False)
pupilData_filtered['pupilRight_hampel'] = hampel(pupilData_woBlinks['pupilRight'], 25, 3, False)

pupilData_filtered['timeStamp_plot'] = [time/1000 for time in pupilData_filtered['timeStamp']]

blinks being combined


In [165]:
fig = plt.figure()

ax = fig.add_subplot(111)

#ax.plot(timeMilliS, pupilRawLeft, label = 'with Nan')
ax.plot(pupilData_filtered['timeStamp_plot'], pupilRawLeft, label = 'unfiltered left')
ax.plot(pupilData_filtered['timeStamp_plot'], pupilData_filtered['pupilLeft'], label = 'filtered left')

#ax.plot(pupilData_woBlinks['timeStamp'], pupilRawRight, '-.', label = 'unfiltered right')
#ax.plot(pupilData_woBlinks['timeStamp'], pupilData_woBlinks['pupilRight'], '-.', label = 'filtered right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Absolute pupil size [in mm]')

ax.set_title('Check blink removal filter for the complete experiment')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName + '\\checkFiltering_leftPupil_blinkRemoval', 'wb'))
#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName + '\\checkFiltering_bothPupil_blinkRemoval', 'wb'))



In [166]:
fig = plt.figure()

ax = fig.add_subplot(111)

#ax.plot(timeMilliS, pupilRawLeft, label = 'with Nan')
ax.plot(pupilData_filtered['timeStamp_plot'], pupilData_filtered['pupilLeft'], label = 'unfiltered')
ax.plot(pupilData_filtered['timeStamp_plot'], pupilData_filtered['pupilLeft_hampel'], '-.', label = 'filtered')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Absolute pupil size [in mm]')

ax.set_title('Check hampel filter for the complete experiment')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName + '\\checkFiltering_leftPupil_hampelFilter', 'wb'))



## Pupil sizes from pin in the dark

In [167]:
# indexes of pin events
index_pin_start =  [ind for ind, row in events_wInd if row == 'pin_start']
index_pin_end = [ind for ind, row in events_wInd if row == 'pin_end']


# pupil sizes for pain from pin 
pupilRawLeft_pin_dark = pupilData_filtered['pupilLeft_hampel'][index_pin_start[0]:index_pin_end[0]]
pupilRawRight_pin_dark = pupilData_filtered['pupilRight_hampel'][index_pin_start[0]:index_pin_end[0]]
time_pin_dark = pupilData_filtered['timeStamp'][index_pin_start[0]:index_pin_end[0]]

time_pin_dark_plot = [time/1000 for time in time_pin_dark]

print('Ratio of nan values in this expt from pin is: ', sum(interpolatedValues[index_pin_start[0]:index_pin_end[0]])/len(interpolatedValues[index_pin_start[0]:index_pin_end[0]]))

Ratio of nan values in this expt from pin is:  0.08593091828138164


In [168]:
# find the logged events in the pin events

for index in indices_loggedEvents_new:
    if index > index_pin_start[0]:
        logged_events_start = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_pin_end[0]:
        logged_events_end = index
        break    
        
indices_loggedEvents_pin_dark = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start):indices_loggedEvents_new.index(logged_events_end)+1]

In [169]:
# plot the filtered signal for pins in the dark and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_pin_dark_plot, pupilRawLeft_pin_dark, label = 'left')
ax.plot(time_pin_dark_plot, pupilRawRight_pin_dark, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the dark' %subjectName)

for ind in indices_loggedEvents_pin_dark:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_pin_dark), np.max(pupilRawLeft_pin_dark)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_pin_dark_wLogEventsMarked', 'wb'))


## Average of individual trials

In [170]:
# average of pin trials

pinTrials_dark_start = indices_loggedEvents_pin_dark[0:-1]
pinTrials_dark_end = indices_loggedEvents_pin_dark[1:]

pinTrial_dark_1 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_dark_start[0]:pinTrials_dark_end[0]].tolist()
pinTrial_dark_2 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_dark_start[1]:pinTrials_dark_end[1]].tolist()
pinTrial_dark_3 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_dark_start[2]:pinTrials_dark_end[2]].tolist()
pinTrial_dark_4 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_dark_start[3]:pinTrials_dark_end[3]].tolist()

timeTrial_dark_1 = np.arange(0, len(pinTrial_dark_1)/100, 1/100)
timeTrial_dark_2 = np.arange(0, len(pinTrial_dark_2)/100, 1/100)
timeTrial_dark_3 = np.arange(0, len(pinTrial_dark_3)/100, 1/100)
timeTrial_dark_4 = np.arange(0, len(pinTrial_dark_4)/100, 1/100)

trialLength_avg_min = np.min([len(pinTrial_dark_1), len(pinTrial_dark_2), len(pinTrial_dark_3), len(pinTrial_dark_4)])

trialLength_avg_minInd = np.argmin([len(pinTrial_dark_1), len(pinTrial_dark_2), len(pinTrial_dark_3), len(pinTrial_dark_4)])

if trialLength_avg_minInd == 0:
    timeTrial_dark_avgPlot = timeTrial_dark_1
elif trialLength_avg_minInd == 1:
    timeTrial_dark_avgPlot = timeTrial_dark_2
elif trialLength_avg_minInd == 2:
    timeTrial_dark_avgPlot = timeTrial_dark_3
else:
    timeTrial_dark_avgPlot = timeTrial_dark_4

pinTrial_dark_avg = np.average([pinTrial_dark_1[0:trialLength_avg_min], pinTrial_dark_2[0:trialLength_avg_min], pinTrial_dark_3[0:trialLength_avg_min], pinTrial_dark_4[0:trialLength_avg_min]], axis = 0)
pinTrial_dark_std = np.std([pinTrial_dark_1[0:trialLength_avg_min], pinTrial_dark_2[0:trialLength_avg_min], pinTrial_dark_3[0:trialLength_avg_min], pinTrial_dark_4[0:trialLength_avg_min]], axis = 0)
pinTrial_dark_stdError = (np.std([pinTrial_dark_1[0:trialLength_avg_min], pinTrial_dark_2[0:trialLength_avg_min], pinTrial_dark_3[0:trialLength_avg_min], pinTrial_dark_4[0:trialLength_avg_min]], axis = 0))/np.sqrt(4)



In [171]:


fig = plt.figure()

ax = fig.add_subplot(111)
ax.plot(timeTrial_dark_1, pinTrial_dark_1, label = 'trial2')
ax.plot(timeTrial_dark_2, pinTrial_dark_2, label = 'trial3')
ax.plot(timeTrial_dark_3[0:len(pinTrial_dark_3)], pinTrial_dark_3, label = 'trial4')
ax.plot(timeTrial_dark_4, pinTrial_dark_4, label = 'trial5')
ax.plot(timeTrial_dark_avgPlot, pinTrial_dark_avg, '--', label = 'mean' )
ax.plot(timeTrial_dark_avgPlot, pinTrial_dark_avg+pinTrial_dark_stdError, ':', label = 'error')
ax.plot(timeTrial_dark_avgPlot, pinTrial_dark_avg-pinTrial_dark_stdError, ':', label = 'error')

ax.fill_between(timeTrial_dark_avgPlot, pinTrial_dark_avg, pinTrial_dark_avg+pinTrial_dark_stdError, color = 'yellow')
ax.fill_between(timeTrial_dark_avgPlot, pinTrial_dark_avg, pinTrial_dark_avg-pinTrial_dark_stdError, color = 'khaki')


plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the dark' %subjectName)

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_pin_dark_trialsAvg_wError', 'wb'))



## Pupil sizes from hot cup in the dark

In [172]:
# indexes for the pain using hot cup in the dark

# indexes of hot events
index_hot_start =  [ind for ind, row in events_wInd if row == 'hot_start']
index_hot_end = [ind for ind, row in events_wInd if row == 'hot_end']


# hand1
# indexes of hot events for hand1
index_hot_start_H1 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '1'][0]
index_hot_end_H1 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '1'][0]


# pupil sizes for pain from hot for hand1
pupilRawLeft_hot_dark_H1 = pupilData_filtered['pupilLeft_hampel'][index_hot_start_H1:index_hot_end_H1]
pupilRawRight_hot_dark_H1 = pupilData_filtered['pupilRight_hampel'][index_hot_start_H1:index_hot_end_H1]
time_hot_dark_H1 = pupilData_filtered['timeStamp'][index_hot_start_H1:index_hot_end_H1]

time_hot_dark_plot_H1 = [time/1000 for time in time_hot_dark_H1]

# hand2
# indexes of hot events for hand2
index_hot_start_H2 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '2'][0]
index_hot_end_H2 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '2'][0]


# pupil sizes for pain from hot for hand2
pupilRawLeft_hot_dark_H2 = pupilData_filtered['pupilLeft_hampel'][index_hot_start_H2:index_hot_end_H2]
pupilRawRight_hot_dark_H2 = pupilData_filtered['pupilRight_hampel'][index_hot_start_H2:index_hot_end_H2]
time_hot_dark_H2 = pupilData_filtered['timeStamp'][index_hot_start_H2:index_hot_end_H2]

time_hot_dark_plot_H2 = [time/1000 for time in time_hot_dark_H2]

print('ratio of nan values during the hot cup in the dark with the first hand is: ', sum(interpolatedValues[index_hot_start_H1:index_hot_end_H1])/len(interpolatedValues[index_hot_start_H1:index_hot_end_H1]))

print('ratio of nan values during the hot cup in the dark with the second hand is: ', sum(interpolatedValues[index_hot_start_H2:index_hot_end_H2])/len(interpolatedValues[index_hot_start_H2:index_hot_end_H2]))


ratio of nan values during the hot cup in the dark with the first hand is:  0.5790190735694822
ratio of nan values during the hot cup in the dark with the second hand is:  0.518628912071535


In [173]:
# find the logged events in the hot events in the dark

# hand1
for index in indices_loggedEvents_new:
    if index > index_hot_start_H1:
        logged_events_start_H1 = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_hot_end_H1:
        logged_events_end_H1 = index
        break

indices_loggedEvents_hot_dark_H1 = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start_H1):indices_loggedEvents_new.index(logged_events_end_H1)+1]


# hand2
for index in indices_loggedEvents_new:
    if index > index_hot_start_H2:
        logged_events_start_H2 = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_hot_end_H2:
        logged_events_end_H2 = index
        break
        
indices_loggedEvents_hot_dark_H2 = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start_H2):indices_loggedEvents_new.index(logged_events_end_H2)+1]


In [174]:
# plot the filtered signal for hot cup for hand1 in the dark and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_dark_plot_H1, pupilRawLeft_hot_dark_H1, label = 'left')
ax.plot(time_hot_dark_plot_H1, pupilRawRight_hot_dark_H1, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand1 in the dark' %subjectName)

for ind in indices_loggedEvents_hot_dark_H1:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_hot_dark_H1), np.max(pupilRawLeft_hot_dark_H1)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_dark_wLogEventsMarked_H1', 'wb'))


In [175]:
# plot the filtered signal for hot cup for hand2 in the dark and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_dark_plot_H2, pupilRawLeft_hot_dark_H2, label = 'left')
ax.plot(time_hot_dark_plot_H2, pupilRawRight_hot_dark_H2, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand2 in the dark' %subjectName)

for ind in indices_loggedEvents_hot_dark_H2:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_hot_dark_H2), np.max(pupilRawLeft_hot_dark_H2)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_dark_wLogEventsMarked_H2', 'wb'))


## Pupil sizes from pins in light

In [176]:

# pupil sizes for pain from pin 
pupilRawLeft_pin_light = pupilData_filtered['pupilLeft_hampel'][index_pin_start[-1]:index_pin_end[-1]]
pupilRawRight_pin_light = pupilData_filtered['pupilRight_hampel'][index_pin_start[-1]:index_pin_end[-1]]
time_pin_light = pupilData_filtered['timeStamp'][index_pin_start[-1]:index_pin_end[-1]]


time_pin_light_plot = [time/1000 for time in time_pin_light]

print('Ratio of nan values in this expt from pin is: ', sum(interpolatedValues[index_pin_start[-1]:index_pin_end[-1]])/len(interpolatedValues[index_pin_start[-1]:index_pin_end[-1]]))

Ratio of nan values in this expt from pin is:  0.04774241302738712


In [177]:
# find the logged events in the pin events

for index in indices_loggedEvents_new:
    if index > index_pin_start[-1]:
        logged_events_start = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_pin_end[-1]:
        logged_events_end = index
        break
        
indices_loggedEvents_pin_light = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start):indices_loggedEvents_new.index(logged_events_end)+1]

In [178]:
# plot the filtered signal for pins in the light and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_pin_light_plot, pupilRawLeft_pin_light, label = 'left')
ax.plot(time_pin_light_plot, pupilRawRight_pin_light, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the light' %subjectName)

for ind in indices_loggedEvents_pin_light:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_pin_light), np.max(pupilRawLeft_pin_light)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_pin_light_wLogEventsMarked', 'wb'))


In [179]:
# average of pin trials

pinTrials_light_start = indices_loggedEvents_pin_light[0:-1]
pinTrials_light_end = indices_loggedEvents_pin_light[1:]

pinTrial_light_1 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_light_start[0]:pinTrials_light_end[0]]
pinTrial_light_2 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_light_start[1]:pinTrials_light_end[1]].tolist()
pinTrial_light_3 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_light_start[2]:pinTrials_light_end[2]].tolist()
pinTrial_light_4 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_light_start[3]:pinTrials_light_end[3]].tolist()

timeTrial_light_1 = np.arange(0, len(pinTrial_light_1)/100, 1/100)
timeTrial_light_2 = np.arange(0, len(pinTrial_light_2)/100, 1/100)
timeTrial_light_3 = np.arange(0, len(pinTrial_light_3)/100, 1/100)
timeTrial_light_4 = np.arange(0, len(pinTrial_light_4)/100, 1/100)


trialLength_avg_min = np.min([len(pinTrial_light_1), len(pinTrial_light_2), len(pinTrial_light_3), len(pinTrial_light_4)])

trialLength_avg_minInd = np.argmin([len(pinTrial_light_1), len(pinTrial_light_2), len(pinTrial_light_3), len(pinTrial_light_4)])

if trialLength_avg_minInd == 0:
    timeTrial_light_avgPlot = timeTrial_light_1
elif trialLength_avg_minInd == 1:
    timeTrial_light_avgPlot = timeTrial_light_2
elif trialLength_avg_minInd == 2:
    timeTrial_light_avgPlot = timeTrial_light_3
else:
    timeTrial_light_avgPlot = timeTrial_light_4
    
pinTrial_light_avg = np.average([pinTrial_light_1[0:trialLength_avg_min], pinTrial_light_2[0:trialLength_avg_min], pinTrial_light_3[0:trialLength_avg_min], pinTrial_light_4[0:trialLength_avg_min]], axis = 0)
pinTrial_light_std = np.std([pinTrial_light_1[0:trialLength_avg_min], pinTrial_light_2[0:trialLength_avg_min], pinTrial_light_3[0:trialLength_avg_min], pinTrial_light_4[0:trialLength_avg_min]], axis = 0)
pinTrial_light_stdError = (np.std([pinTrial_light_1[0:trialLength_avg_min], pinTrial_light_2[0:trialLength_avg_min], pinTrial_light_3[0:trialLength_avg_min], pinTrial_light_4[0:trialLength_avg_min]], axis = 0))/np.sqrt(4)



In [180]:


fig = plt.figure()

ax = fig.add_subplot(111)
ax.plot(timeTrial_light_1, pinTrial_light_1, label = 'trial2')
ax.plot(timeTrial_light_2, pinTrial_light_2, label = 'trial3')
ax.plot(timeTrial_light_3, pinTrial_light_3, label = 'trial4')
ax.plot(timeTrial_light_4, pinTrial_light_4, label = 'trial5')
ax.plot(timeTrial_light_avgPlot, pinTrial_light_avg, '--', label = 'mean' )
ax.plot(timeTrial_light_avgPlot, pinTrial_light_avg+pinTrial_light_stdError, ':', label = 'error')
ax.plot(timeTrial_light_avgPlot, pinTrial_light_avg-pinTrial_light_stdError, ':', label = 'error')

ax.fill_between(timeTrial_light_avgPlot, pinTrial_light_avg, pinTrial_light_avg+pinTrial_light_stdError, color = 'yellow')
ax.fill_between(timeTrial_light_avgPlot, pinTrial_light_avg, pinTrial_light_avg-pinTrial_light_stdError, color = 'khaki')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the light' %subjectName)

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_pin_light_trialsAvg_wError', 'wb'))



## Pupil sizes from hot cup in the light

In [181]:
# indexes for the pain using hot cup in the light

# indexes of hot events
index_hot_start =  [ind for ind, row in events_wInd if row == 'hot_start']
index_hot_end = [ind for ind, row in events_wInd if row == 'hot_end']


# hand1
# indexes of hot events for hand1
index_hot_start_H1 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '1'][-1]
index_hot_end_H1 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '1'][-1]

# pupil sizes for pain from hot for hand1
pupilRawLeft_hot_light_H1 = pupilData_filtered['pupilLeft_hampel'][index_hot_start_H1:index_hot_end_H1]
pupilRawRight_hot_light_H1 = pupilData_filtered['pupilRight_hampel'][index_hot_start_H1:index_hot_end_H1]
time_hot_light_H1 = pupilData_filtered['timeStamp'][index_hot_start_H1:index_hot_end_H1]


time_hot_light_plot_H1 = [time/1000 for time in time_hot_light_H1]


# hand2
# indexes of hot events for hand2
index_hot_start_H2 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '2'][-1]
index_hot_end_H2 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '2'][-1]

# pupil sizes for pain from hot for hand2
pupilRawLeft_hot_light_H2 = pupilData_filtered['pupilLeft_hampel'][index_hot_start_H2:index_hot_end_H2]
pupilRawRight_hot_light_H2 = pupilData_filtered['pupilRight_hampel'][index_hot_start_H2:index_hot_end_H2]
time_hot_light_H2 = pupilData_filtered['timeStamp'][index_hot_start_H2:index_hot_end_H2]

time_hot_light_plot_H2 = [time/1000 for time in time_hot_light_H2]


print('ratio of nan values during the hot cup in the light with the first hand is: ', sum(interpolatedValues[index_hot_start_H1:index_hot_end_H1])/len(interpolatedValues[index_hot_start_H1:index_hot_end_H1]))

print('ratio of nan values during the hot cup in the light with the second hand is: ', sum(interpolatedValues[index_hot_start_H2:index_hot_end_H2])/len(interpolatedValues[index_hot_start_H2:index_hot_end_H2]))


ratio of nan values during the hot cup in the light with the first hand is:  0.40298507462686567
ratio of nan values during the hot cup in the light with the second hand is:  0.5410071942446043


In [182]:
# find the logged events in the hot events in the light

# hand1
for index in indices_loggedEvents_new:
    if index > index_hot_start_H1:
        logged_events_start_H1 = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_hot_end_H1:
        logged_events_end_H1 = index
        break

indices_loggedEvents_hot_light_H1 = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start_H1):indices_loggedEvents_new.index(logged_events_end_H1)+1]


# hand2
for index in indices_loggedEvents_new:
    if index > index_hot_start_H2:
        logged_events_start_H2 = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_hot_end_H2:
        logged_events_end_H2 = index
        break
        
indices_loggedEvents_hot_light_H2 = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start_H2):indices_loggedEvents_new.index(logged_events_end_H2)+1]


In [183]:
# plot the filtered signal for hot cup for hand1 in the light and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_light_plot_H1, pupilRawLeft_hot_light_H1, label = 'left')
ax.plot(time_hot_light_plot_H1, pupilRawRight_hot_light_H1, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand1 in the light' %subjectName)

for ind in indices_loggedEvents_hot_light_H1:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_hot_light_H1), np.max(pupilRawRight_hot_light_H1)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_light_wLogEventsMarked_H1', 'wb'))


In [184]:
# plot the filtered signal for hot cup for hand2 in the light and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_light_plot_H2, pupilRawLeft_hot_light_H2, label = 'left')
ax.plot(time_hot_light_plot_H2, pupilRawRight_hot_light_H2, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand2 in the light' %subjectName)

for ind in indices_loggedEvents_hot_light_H2:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_hot_light_H2), np.max(pupilRawLeft_hot_light_H2)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_light_wLogEventsMarked_H2', 'wb'))


In [222]:
def lowPassFilter(pupilSeries):
    
    
    FreqCutoff = 0.5
    FreqNyquist = float(100/2)
    b, a = signal.butter(8, float(FreqCutoff/FreqNyquist), btype='highpass')

    pupilArray = np.array(pupilSeries.tolist())
    pupilLPF = signal.filtfilt(b, a, pupilArray)
    
    
    return pupilLPF

In [223]:
pupilData_filtered['pupilLeft_lpf'] = lowPassFilter(pupilData_filtered['pupilLeft_hampel'])

plt.plot(pupilData_filtered['timeStamp'], pupilData_filtered['pupilLeft_hampel'])
plt.plot(pupilData_filtered['timeStamp'], pupilData_filtered['pupilLeft_lpf'])

In [224]:
t = np.arange(len(pupilRawRight_pin_dark))
sp = np.fft.fft(pupilRawLeft_pin_dark)
freq = np.fft.fftfreq(t.shape[-1])
plt.figure()
plt.plot(freq, sp.real)
plt.show()